In [1]:
"""
Author : Abhinav S. Thorat 
Date : 27 Nov 2020
Updated : 16 Feb 2022
"""

'\nAuthor : Abhinav S. Thorat \nDate : 27 Nov 2020\nUpdated : 16 Feb 2022\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
# IGNORE THE CONTENT OF THIS CELL
# tf.compat.v1.disable_eager_execution()

In [4]:
tf.__version__

'2.5.0'

In [5]:
## Train Keep it False , For training train == True

train = False

## Step 1: The Data



In [6]:
path_to_file = 'shakespeare.txt'

In [7]:
text = open(path_to_file, 'r').read()

In [8]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


### Understanding unique characters

In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

## Step 2: Text Processing

### Text Vectorization

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

In [10]:
char_to_ind = {u:i for i, u in enumerate(vocab)}

In [11]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [12]:
ind_to_char = np.array(vocab)

In [13]:
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [14]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [15]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

We now have a mapping we can use to go back and forth from characters to numerics.

In [16]:
sample = text[:20]
sample

'\n                   '

In [17]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Step 3: Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In [18]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [19]:
line = "From fairest creatures we desire increase"

In [20]:
len(line)

41

In [21]:
part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [22]:
len(part_stanza)

131

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [23]:
seq_len = 120

In [24]:
total_num_seq = len(text)//(seq_len+1)

In [25]:
total_num_seq

45005

In [26]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [27]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [28]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [29]:
dataset = sequences.map(create_seq_targets)

In [30]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [31]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [32]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int32, tf.int32)>

## Step 4: Creating the Model


The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [33]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True

In [35]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [36]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)
    Computes the sparse categorical crossentropy loss.
    
    Standalone usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    Args:
      y_true: Ground truth values.
      y_pred: The predicted values.
      from_logits: Whether `y_pred` is expected to be a logits tensor. By default,
        we assume that `y_pred` encodes a probability distribution.
      axis: (Optional) Defaults to -1. The dimension along which the entropy is
        computed.
    
    Returns:
      Sparse categorical crossentropy loss value.



In [37]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [38]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [39]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


## Step 5: Training the model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.



In [41]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [42]:
example_batch_predictions

<tf.Tensor: shape=(128, 120, 84), dtype=float32, numpy=
array([[[ 6.65498059e-03, -3.70770879e-03, -2.88563408e-03, ...,
         -7.46741018e-04, -9.24810360e-04, -1.28200266e-03],
        [ 3.58518399e-03, -2.39627971e-03, -4.16898605e-04, ...,
         -3.35837901e-03,  6.26634108e-03, -2.06089811e-03],
        [ 6.76885573e-03, -3.48366797e-03,  6.99417247e-03, ...,
         -2.76751816e-03,  7.76957162e-03, -3.48676112e-03],
        ...,
        [ 1.67419913e-03, -5.53035270e-03, -6.01145846e-04, ...,
         -5.15966909e-03, -2.52451934e-03, -8.04253574e-03],
        [ 1.45218160e-03, -3.96687258e-03, -3.41172330e-03, ...,
         -1.12127536e-03,  1.85203133e-03, -6.31896267e-03],
        [ 4.71617002e-03, -4.36192611e-03,  5.06340200e-03, ...,
         -2.56591593e-04,  4.40026028e-03, -4.39770473e-03]],

       [[ 4.99445014e-03, -2.04369705e-03,  6.13579946e-03, ...,
         -1.17957906e-03,  3.90448933e-03, -2.32380629e-03],
        [-6.92296599e-04, -1.31281270e-02,  3.5

In [43]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [44]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[36],
       [59],
       [42],
       [54],
       [39],
       [36],
       [ 5],
       [74],
       [45],
       [62],
       [44],
       [32],
       [49],
       [27],
       [23],
       [56],
       [68],
       [52],
       [16],
       [83],
       [66],
       [15],
       [44],
       [52],
       [46],
       [68],
       [22],
       [ 7],
       [22],
       [28],
       [23],
       [48],
       [59],
       [37],
       [ 7],
       [61],
       [44],
       [26],
       [31],
       [46],
       [ 8],
       [67],
       [50],
       [12],
       [66],
       [25],
       [28],
       [13],
       [35],
       [41],
       [77],
       [48],
       [30],
       [46],
       [31],
       [53],
       [23],
       [21],
       [65],
       [46],
       [51],
       [ 2],
       [ 0],
       [10],
       [ 4],
       [57],
       [30],
       [45],
       [66],
       [ 7],
       [73],
       [55],
       [82],
   

In [45]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [46]:
sampled_indices

array([36, 59, 42, 54, 39, 36,  5, 74, 45, 62, 44, 32, 49, 27, 23, 56, 68,
       52, 16, 83, 66, 15, 44, 52, 46, 68, 22,  7, 22, 28, 23, 48, 59, 37,
        7, 61, 44, 26, 31, 46,  8, 67, 50, 12, 66, 25, 28, 13, 35, 41, 77,
       48, 30, 46, 31, 53, 23, 21, 65, 46, 51,  2,  0, 10,  4, 57, 30, 45,
       66,  7, 73, 55, 82, 68,  9, 10, 29, 38, 45, 66, 63, 11,  3, 38, 47,
       78, 78, 53, 59,  5, 16, 22, 50, 30, 65, 67, 19, 30, 45, 26, 43, 69,
       74, 10, 41, 39, 36, 23, 19, 27, 56, 50, 33, 64,  1, 10, 27, 81, 63,
       39], dtype=int64)

In [47]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


Given the input seq: 

not to cut would show thee but a fool,
    Since I have ever followed thee with hate,  
    Drawn tuns of blood out of t


Next Char Predictions: 

KdQ_NK'sTgSGXB<am[5}k4S[Um;);C<WdL)fSAFU,lY1k?C2JPvWEUF]<:jUZ!
.&bETk)r`|m-.DMTkh0"MVww]d'5;YEjl8ETARns.PNK<8BaYHi .BzhN


After confirming the dimensions are working, let's train our network!

In [48]:
epochs = 30

In [49]:
from tensorflow.keras.models import load_model

In [50]:
if train:
    model.fit(dataset,epochs=epochs)
    model.save('shakespeare_gen.h5') ### Change File name with training 
else:
    model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

    model.load_weights('shakespeare_gen.h5')

    model.build(tf.TensorShape([1, None]))

## Step 6: Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [52]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [53]:
print(generate_text(model,"flower",gen_size=1000))

flower.
  Tenth not to be believ'd;
    Present to me, hide thy former show,
    hou say he waits on this pennyworth object of it
    Murd'red his part between the child such city must not
    such a blind man keep his book old's gainer of is.
    O wondrous house let me have your heart, stand passing;
    That you are touch'd with prospect of yest
    in earthly miserable stars i' till now fall
    In the hazardon of Laertline be entreated that
    Men at dinner sign'd on my belly already;
    I love herself, and wept for kiss you. Marry, sir, truth is
    embasted. He hath a merciful grave, poor bald shield on a
    wrightio. Brother, thou art; the fool go hence a dream.
  GLOUCESTER. Fie, fie, how immort? I have been a scourge a
    jewel always of it came from the mom him s]
    Enseal thy dear boy's ain, to take the grass
    Excuse that household occasion
    In restless day is to this plaxen
    Onlysic here at the water. Proceed in the night;
    because will be married to her 

In [54]:
print(generate_text(model,"JULIET",gen_size=1000))

JULIETHESSEDASTRIA. Canters                   [Exeunt Duncain Tebulian trung  
    Of lust have tears seen against the worse.
    The iram as ballow'd, his his allegians,
    Whose unto his sake
    May, lust, and ripen!
  LOOND. Put so mean'st that house, Sir Robert Sweeses,
    Patient the guess-weight.
  PROSPERO. For my myster gave her tradling whom I wish
    These foretains of purposed.
  QUEEN ELIZABETH. Then two young MISTRESS FORD
  MARCUS. As I am constant when the Grecian confesses!
  FERSTNIA. Break too be here to-day, sir; here he wonth
    The several count.

  URTINIO. Sir, tell me wherefore I desire all!
  JULIA.  [Aside]  The Cardinal confession of a sheep,
    Not with such cansumellogs of folly,
    As thus forg'd it that I took together.
  HELENA. Ay, if my brother I had to our kiss;
    This will serve an Gone, tyrangs sauch'd affectures-
    a sad cannon; but countenance me the villainies,
    Cry letters, flattering of your warm,
    Subdient like great assays hi